# Чтение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [4]:
tags = pd.read_csv('data/tags.csv', sep=',#', names=['tag', 'name', 'type'])
tags = tags.loc[tags['type'] == '"1"']
tags.head(10)

,tag,name,type
151,"""179""","""О войне""","""1"""
152,"""180""","""военный""","""1"""
153,"""181""","""заграничная поездка""","""1"""
154,"""182""","""блокада""","""1"""
155,"""183""","""тыл""","""1"""
156,"""184""","""фронтовой""","""1"""
157,"""185""","""депортация""","""1"""
158,"""186""","""детский""","""1"""
159,"""187""","""профессиональный""","""1"""
160,"""188""","""путешествие""","""1"""


Всего размечено 86 тем. Из таблицы можно заметить, что есть темы, которые пересекаются, например "О войне", "военный", "фронтовый", "тыл".

In [5]:
tags.shape

(86, 3)

In [6]:
# список тем типа 1
themes = list(tags['tag'])

In [7]:
diaries = pd.read_csv('data/diary.csv', sep=',#', header=None)
diaries.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"""3""","""4""","""published""","""329""","""1932-09-28""","""1937-04-30""",NaN,"""0""","""0""","""0""","""1425446154"""
1,"""4""","""5""","""published""","""356""","""1942-09-09""","""1944-01-27""",NaN,"""0""","""0""","""1474033861""","""1425446228"""
2,"""5""","""6""","""published""","""28""","""1943-01-31""","""1951-07-03""",NaN,"""0""","""0""","""0""","""1425446370"""
3,"""6""","""7""","""published""","""65""","""0000-00-00""","""0000-00-00""",NaN,"""0""","""0""","""0""","""1425446645"""
4,"""8""","""11""","""published""","""804""","""1943-08-04""","""1961-12-31""",NaN,"""0""","""0""","""0""","""1425487703"""


Всего есть 2342 дневника.

In [8]:
len(diaries[0])

2342

In [9]:
tags_diaries = pd.read_csv('data/tags_diaries.csv', sep=',#', names=['diary', 'tag'])
tags_diaries.head()

,diary,tag
0,"""3""","""181"""
1,"""3""","""328"""
2,"""4""","""180"""
3,"""4""","""184"""
4,"""8""","""187"""


Из них хоть какой-то тэг приписан к 1407 дневникам. К одному дневнику может быть приписано несколько тэгов.

In [10]:
len(set(tags_diaries['diary']))

1407

In [11]:
# датафрейм с id дневников, которым приписана хотя бы одна тема
diaries_with_themes = tags_diaries[tags_diaries['tag'].isin(themes)]

Тема приписана 1407 дневникам. Многим дневникам приписано несколько тем. Видимо, потому, что некоторое темы пересекаются или у одного дневника может быть несколько разных тем. 

Как можно заметить, локации и персоны ни одному дневнику не приписаны. 

In [12]:
len(set(diaries_with_themes['diary']))

1407

In [13]:
with open('data/notes.json', 'r') as f:
        notes = json.load(f)

В джейсоне 182975 записи (кажется, в csv было больше).

In [14]:
len(notes[2]['data'])

182975

In [26]:
notes[2]['data'][-1]

{'createdDate': '1485536357',
 'date': '2002-03-18',
 'dateTop': '0000-00-00',
 'diary': '145',
 'id': '221877',
 'julian_calendar': '0',
 'notDated': '0',
 'pictures': '1',
 'text': 'Впервые в жизни получил какое-то удовольствие от тренажера. Нынче он у нас стал компьютерным, цветным, адекватным. Не все, конечно, еще отлажено, но против старого — небо и земля. На нем можно решать задачи. С пожаром на взлете мы приземлились за 2.45, визуально, ну, с чуть взмокшей спиной. Так это ж я, старик; молодые-то быстрее зайдут.    \nПриехала газетчица Маша; дождался, посмотрел материал. От всего разговора, в статью вошли, конечно, крохи, и не самые лучшие; пришлось править, кое-что даже принципиально. Для газетенки сойдет. Сказала, читайте послезавтра в газете… Кофейком ее напоил… но все как-то на скоростях. Оно и понятно: репортеры — они как хариусы: хвать на лету — и дальше…    \nЧем мне, собственно, не понравился материал? Тем, что в этом интервью о моей большой книге — ни слова. Вообще, на м

In [15]:
rows = []

for note in notes[2]['data']:
    rows.append([note['text'], note['diary']])

In [16]:
notes = pd.DataFrame(rows, columns=['text', 'diary'])

In [17]:
# датафрейм с записями и id их дневника
notes.head()

,text,diary
0,Барабинск. Татарка.,130
1,Омск. Утро. Стоим до вечера. Обед. Сухой паек.,130
2,В 6 часов утра еще темно. Станция Называевская...,130
3,Утром рано приехали в Тюкалинск. Из автоотряда...,130
4,"Так давно я не прикасался к дневнику, что он м...",234


# Предобработка

In [18]:
from stop_words import get_stop_words
import nltk
from pymystem3 import Mystem
import re

In [19]:
random_seed = 42

In [20]:
stop_words = get_stop_words('ru')
stop_words.append('свой')

In [21]:
m = Mystem()

In [22]:
def tokenize(text):
    comp = re.compile('[А-ЯЁа-яё\-]+')
    tokens = comp.findall(text.lower())
    return tokens

In [23]:
def lemmatize(text):
    return [m.lemmatize(word)[0] for word in text]

In [52]:
# токенизируем заметки
notes_lemma = pd.DataFrame()
notes_lemma['text'] = notes.text[:10000].apply(lambda x: tokenize(x))

In [53]:
# лемматизируем заметки
notes_lemma['text'] = notes_lemma.text.apply(lambda x: lemmatize(x))

In [66]:
notes_lemma = notes_lemma.text.apply(lambda x: ' '.join(x))

In [67]:
notes_lemma.head(10)

0                                    барабинск татарка
1            омск утро стоять до вечер обед сухой паек
2    в час утро еще темно станция называевский слез...
3    утро рано приезжать в тюкалинск из автоотряд и...
4    так давно я не прикасаться к дневник что он я ...
5    еще проходить пешком более км навстречу выходи...
6    стоять на место за это время поступать несколь...
7    приезжать на станция потом идти до военный гор...
8    мы отделять от рот в отдельный комната вчера б...
9    мы перевозить в расположение бригада где сформ...
Name: text, dtype: object

# Тематическое моделирование

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(notes_lemma)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [69]:
from sklearn.decomposition import NMF

In [62]:
no_topics = 10

# NMF
nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [63]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [70]:
no_top_words = 20
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
говорить дело знать русский давать становиться москва самый идти вчера оставаться видеть большой думать большевик последний работа место рассказывать новый
Topic 1:
мао кпк цзэ цк китай ван чан председатель кай партия кан гоминьдан район китайский руководство шэн советский особый политический яньань
Topic 2:
язык болгарский статья славянский институт доклад словарь языкознание атлас история славяноведение работа сравнительный вопрос ученый печать написать грамматика должный сборник
Topic 3:
немец фронт немецкий армия война ленинград противник дивизия бой удар город гитлер наступление войско идти красный ленинградский самолет германия район
Topic 4:
тов отряд федоров рота эшелон командир группа балицкий елезный паровоз район взвод партизан вагон подорвать соединение штаб радиограмма вражеский разведка
Topic 5:
японец снаряд батарея японский генерал час гора город огонь форт позиция стрелять гавань фланг орудие стессель миноносец крепость артур судно
Topic 6:
миша пьеса театр ве

In [71]:
def get_topics(texts):
    
    notes_lemma = pd.DataFrame()
    notes_lemma['text'] = texts.apply(lambda x: tokenize(x))
    notes_lemma['text'] = notes_lemma.text.apply(lambda x: lemmatize(x))
    notes_lemma = notes_lemma.text.apply(lambda x: ' '.join(x))
    
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
    tfidf = tfidf_vectorizer.fit_transform(notes_lemma)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    
    no_topics = 10
    nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, 
              l1_ratio=.5, init='nndsvd').fit(tfidf)
    
    no_top_words = 20
    display_topics(nmf, tfidf_feature_names, no_top_words)

In [72]:
get_topics(notes.text[10000:20000])

Topic 0:
говорить знать становиться жить дело давать работа идти писать час вчера видеть самый работать приходить получать оставаться ребенок думать стоять
Topic 1:
утро арт дождь погода пальба пасмурно стрельба вечер ночь слышный тихо выстрел редкий облачность одиночный ветер слышно идти отдаленный снег
Topic 2:
мина тревога отбой зенитка воздушный возд тихо небо бомба ночь налет слышный облако взрыв объявлять давать звездный сирена мороз бить
Topic 3:
спектакль роль играть театр отелло актер пьеса сцена акт зал маскарад хороший зритель искусство говорить сила делать образ арбенин лира
Topic 4:
немец русский россия война народ правительство советский армия войско немецкий власть фронт солдат мир германия совет большевик москва военный революция
Topic 5:
город фронт враг утро перемена температура самолет орудие ночь налет улица спокойно выстрел обстреливать мороз воздушный солнце течение облако канонада
Topic 6:
полк красный батарея командир взвод батальон орудие гаубица офицер огонь п

In [73]:
get_topics(notes.text[-10000:])

Topic 0:
вечер мамочка письмо час утро получать работа ночь школа идти говорить ходить спать работать приходить обед писать хороший знать проходить
Topic 1:
улей пчела мед выставлять матка пуд погода убирать фунт магазин рой рамка теплый роить остальной зимовка поставлять греча дождик штука
Topic 2:
вопрос москва война ссср англия иден советский германия фронт дело армия правительство швеция черчилль гитлер страна отношение союз политика переговоры
Topic 3:
полет летать экипаж скорость посадка пилот машина самолет садиться полоса рейс командир высота давать метр заход двигатель глиссада лететь газ
Topic 4:
снег выпадать морозец погода подмораживать растаивать оттепель зима теплый стоять санный морозный метель вьюга путь ясный обильный морозно декабрь снежок
Topic 5:
митрополит церковь дарить владыка служить служба воскресение русский архиепископ народ николай храм литургия суббота молебен приход священник епископ церковный св
Topic 6:
андреевна софья львовна александра письмо говорить 

# Записи без тематических тэгов

У 104184 записи нет тематического тэга.

In [45]:
notes_without_themes = notes[~notes['diary'].isin(diaries_with_themes['diary'].apply(lambda x: x.strip('"')))]

In [74]:
notes_without_themes.shape

(104184, 2)

In [75]:
get_topics(notes_without_themes.text[:10000])

Topic 0:
говорить дело русский россия знать становиться война новый народ давать самый мир жить идти видеть думать слово писать большевик вчера
Topic 1:
аля час обед алена веранда копель сидеть фира завтрак море дома копа солнце катя вечер али спать небо сон записывать
Topic 2:
спектакль играть роль театр отелло актер сцена акт зал маскарад хороший зритель сила говорить искусство делать смотреть лира образ получаться
Topic 3:
роман читать ангарский вечер глава вильямс воланд свободный работа написать ермолинский писать править писатель чтение сценарий маргарита слушать коля прийти
Topic 4:
вечер утро час университет читать книга егоров петр богоявленский прогулка заседание академия лекция готье среда любавский четверг пятница работа суббота
Topic 5:
мама письмо милый вечер пяст ремизов люба обедать милая господь тетя терещенко телефон женя приходить ночь сирин писать утро приезжать
Topic 6:
пьеса миша театр вечер мхат звонок дмитриев оля звонить позвонить говорить приходить пойти прийт

In [76]:
get_topics(notes_without_themes.text[-10000:])

Topic 0:
говорить знать вечер думать час хороший становиться москва дело идти любить работа понимать жить самый приходить писать оставаться давать видеть
Topic 1:
читать история доклад ключевский прочитывать заметка кончать статья начинать конспектировать книга глава просматривать заседание заниматься русский исторический вечер наука делать
Topic 2:
улей пчела мед выставлять погода дождь матка пуд убирать магазин фунт рой рамка теплый роить тепло поставлять остальной греча хороший
Topic 3:
работа текущий бсэ библиография библиографический пролетариат история редактировать комиссия статья карточка проредактировать ссср мсэ заседание ленин институт местком написать индекс
Topic 4:
снег тепло дождь мороз погода выпадать река стоять градус теплый лед морозец растаивать становиться зима подмораживать оттепель снежок ясный морозный
Topic 5:
николина горе гора дод уезжать отъезд утро додома учебник выезжать москва вечер работа доклад орг класс вернуться гулять поехать нога
Topic 6:
станица по